In [18]:
import torch
import numpy as np

# 01-Tensors

In [ ]:
x = torch.rand([2, 3])
print(x)
print(x.size())

In [3]:
x = torch.rand(2, 2)
y = torch.rand(2, 2)
print(x)
print(y)

tensor([[0.3773, 0.7637],
        [0.4393, 0.5986]])
tensor([[0.7164, 0.0081],
        [0.5878, 0.5885]])


## Tensors - Operations

In [4]:
z = x + y
z

tensor([[1.0937, 0.7718],
        [1.0271, 1.1871]])

In [6]:
z = torch.add(x, y)
z

tensor([[1.0937, 0.7718],
        [1.0271, 1.1871]])

In [7]:
y.add_(x) # in place addition
y == z

tensor([[True, True],
        [True, True]])

In [8]:
z = torch.sub(x, y)
z

tensor([[-0.7164, -0.0081],
        [-0.5878, -0.5885]])

In [9]:
z = torch.mul(x, y)
z

tensor([[0.4126, 0.5894],
        [0.4512, 0.7106]])

## Tensors - Slicing

In [10]:
x = torch.rand(5, 3)
x

tensor([[0.8132, 0.6795, 0.6702],
        [0.5922, 0.7865, 0.8401],
        [0.8315, 0.7938, 0.0453],
        [0.5934, 0.9114, 0.6274],
        [0.6779, 0.1583, 0.6205]])

In [11]:
x[:, 0] # Only col 0, with all the rows

tensor([0.8132, 0.5922, 0.8315, 0.5934, 0.6779])

In [12]:
x[1, :] # Only row 1, with all cols

tensor([0.5922, 0.7865, 0.8401])

In [ ]:
x[1, 1].item() # Gives the actual value, not a tensor

tensor(0.7865)

## Tensors - Reshaping

In [15]:
x = torch.rand(4, 4)
x

tensor([[0.0369, 0.2352, 0.0439, 0.5040],
        [0.7394, 0.0749, 0.0093, 0.1163],
        [0.4623, 0.1997, 0.3245, 0.6642],
        [0.4750, 0.4993, 0.6056, 0.2409]])

In [ ]:
y = x.view(16)
y # Now it is a 1D vector

tensor([0.0369, 0.2352, 0.0439, 0.5040, 0.7394, 0.0749, 0.0093, 0.1163, 0.4623,
        0.1997, 0.3245, 0.6642, 0.4750, 0.4993, 0.6056, 0.2409])

In [17]:
y = x.view(-1, 8) # Torch does the rest
y

tensor([[0.0369, 0.2352, 0.0439, 0.5040, 0.7394, 0.0749, 0.0093, 0.1163],
        [0.4623, 0.1997, 0.3245, 0.6642, 0.4750, 0.4993, 0.6056, 0.2409]])

## Tensors - Converting from numpy to torch

In [23]:
a = torch.ones(5)
print(type(a))
a

<class 'torch.Tensor'>


tensor([1., 1., 1., 1., 1.])

In [ ]:
b = a.numpy()
print(type(b))
b

# Both element a and b point to the same memory location

<class 'numpy.ndarray'>


array([1., 1., 1., 1., 1.], dtype=float32)

In [25]:
a = np.ones(5)
print(type(a))
a

<class 'numpy.ndarray'>


array([1., 1., 1., 1., 1.])

In [26]:
b = torch.from_numpy(a)
print(type(b))
b

<class 'torch.Tensor'>


tensor([1., 1., 1., 1., 1.], dtype=torch.float64)

In [ ]:
# Need to calculate the gradient later for the optimization step
x = torch.ones(5, requires_grad=True)
x

tensor([1., 1., 1., 1., 1.], requires_grad=True)

# 02-Autograd